# Chatbots

Following: https://python.langchain.com/v0.2/docs/tutorials/chatbot/

In [10]:
from langchain_community.chat_models import ChatOllama

In [11]:
model = ChatOllama(model="llama3")

## General Chat

In [12]:
from langchain_core.messages import HumanMessage

In [13]:
model.invoke([HumanMessage(content="Hello, I'm Oliver")])

AIMessage(content='Nice to meet you, Oliver! How are you today?', response_metadata={'model': 'llama3', 'created_at': '2024-06-05T11:48:25.7356265Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 395067900, 'load_duration': 2556100, 'prompt_eval_count': 10, 'prompt_eval_duration': 168533000, 'eval_count': 13, 'eval_duration': 221897000}, id='run-e413e134-3123-486c-adc9-5139b6705d4a-0')

In [14]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob!', response_metadata={'model': 'llama3', 'created_at': '2024-06-05T11:48:29.8939074Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 390556700, 'load_duration': 1561100, 'prompt_eval_count': 35, 'prompt_eval_duration': 292158000, 'eval_count': 6, 'eval_duration': 94389000}, id='run-379120d8-1000-4395-8488-5e925d990dbf-0')

## Message history

In [20]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables.config import RunnableConfig

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [22]:
config = RunnableConfig(configurable={"session_id": "1337"})

In [23]:
response = with_message_history.invoke(
    [HumanMessage(content="Hello, I'm Oliver")], config=config)

In [24]:
response.content

"Nice to meet you, Oliver! How's your day going so far?"

In [25]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")], config=config)

In [26]:
response.content

'I think I can remember... Your name is Oliver, right?'

# Prompt templates

In [27]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [28]:
response = chain.invoke({"messages": [HumanMessage(content="Hello, I'm Oliver")]})
response.content

"Hello Oliver! It's nice to meet you. Is there something on your mind that you'd like to chat about or ask me? I'm here to help with any questions or topics you'd like to discuss."

And now with history:

In [29]:
with_history = RunnableWithMessageHistory(chain, get_session_history)

In [30]:
config = RunnableConfig(configurable={"session_id": "1338"})

In [31]:
response = with_history.invoke(
    [HumanMessage(content="Hello, I'm Oliver")], config=config)
response.content

"Nice to meet you, Oliver! How can I assist you today? Do you have any questions or topics you'd like to discuss? I'm here to help with any information or advice I can provide. What's on your mind?"

In [33]:
response = with_message_history.invoke(
    [HumanMessage(content="Write a haiku that greets me by my name")],
    config=config,
)

print(response.content)

Oliver's dawn rise
Golden light upon your face
Welcome, dear one


## More template variables

In [34]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [35]:
with_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")